Example usage of FlowClient to run daily location, modal location and flow queries.

## Create an access token
In actual usage, this would be done using [FlowAuth](../flowauth).

In [1]:
import os
import sys
from dotenv import find_dotenv

In [2]:
path_to_utils_module = os.path.join(
    os.path.dirname(find_dotenv()), "..", "integration_tests", "tests"
)
sys.path.insert(0, path_to_utils_module)

In [3]:
import flowclient
from datetime import timedelta
from utils import make_token

In [4]:
claims = {
    "daily_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "modal_location": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2"],
    },
    "flows": {
        "permissions": {"run": True, "poll": True, "get_result": True},
        "spatial_aggregation": ["admin3", "admin2", "admin1"],
    },
}

In [5]:
TOKEN = make_token(
    username="testuser",
    secret_key="secret",
    lifetime=timedelta(days=1),
    claims=claims,
)

## Create an API connection

In [7]:
conn = flowclient.connect("http://localhost:9090", TOKEN)

/Users/jamesharrison/Repositories/FlowKit/flowclient/flowclient/client.py:159: UserWarning: Communications with this server are NOT SECURE.
  return Connection(url, token, api_version, ssl_certificate)


## Daily location

### Specify query parameters

In [8]:
query_spec = flowclient.daily_location(
    date="2016-01-01",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

query_spec

{'query_kind': 'daily_location',
 'params': {'date': '2016-01-01',
  'aggregation_unit': 'admin3',
  'daily_location_method': 'last',
  'subscriber_subset': 'all'}}

### Set query running

In [9]:
query_id = flowclient.run_query(conn, query_spec)
query_id

4a12fae60ce647d01b2bd5902b8a48e7


### Check status of query

In [10]:
flowclient.get_status(conn, query_id)

(True, <Response [303]>)

### Get query result

In [11]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,name,total
0,Dolpa,43
1,Bajhang,17
2,Kavrepalanchok,14
3,Rolpa,12
4,Rukum,14


Alternative (run query, wait for it to finish, and get result, in one function call):

In [12]:
df = flowclient.get_result(conn, query_spec)
df.head()

,name,total
0,Dolpa,43
1,Bajhang,17
2,Kavrepalanchok,14
3,Rolpa,12
4,Rukum,14


## Modal location
### Specify query parameters
Either explicitly from a set of daily locations:

In [13]:
# dl1 = flowclient.daily_location(
#     date="2016-01-01",
#     aggregation_unit="admin3",
#     daily_location_method="last",
#     subscriber_subset=None,
# )
# dl2 = flowclient.daily_location(
#     date="2016-01-02",
#     aggregation_unit="admin3",
#     daily_location_method="last",
#     subscriber_subset=None,
# )
# ml = flowclient.modal_location(dl1, dl2, aggregation_unit="admin3")

# ml

Or from a date range:

In [13]:
ml = flowclient.modal_location_from_dates(
    "2016-01-01",
    "2016-01-03",
    aggregation_unit="admin3",
    daily_location_method="last",
    subscriber_subset=None,
)

ml

{'query_kind': 'modal_location',
 'params': {'locations': ({'query_kind': 'daily_location',
    'params': {'date': '2016-01-01',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-02',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}},
   {'query_kind': 'daily_location',
    'params': {'date': '2016-01-03',
     'aggregation_unit': 'admin3',
     'daily_location_method': 'last',
     'subscriber_subset': 'all'}}),
  'aggregation_unit': 'admin3'}}

### Set query running

In [14]:
query_id = flowclient.run_query(conn, ml)
query_id

64148ac6a93121d8950ea1e70faaf4c8


### Check status of query

In [15]:
flowclient.get_status(conn, query_id)

(True, <Response [303]>)

### Get query result

In [16]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,name,total
0,Dolpa,65
1,Bajhang,13
2,Kavrepalanchok,27
3,Rolpa,17
4,Rukum,12


Alternative:

In [17]:
df = flowclient.get_result(conn, ml)
df.head()

,name,total
0,Dolpa,65
1,Bajhang,13
2,Kavrepalanchok,27
3,Rolpa,17
4,Rukum,12


## Flow
### Specify query parameters
Specify parameters for two location queries:

In [18]:
aggregation_unit = "admin1"
dl_past = flowclient.daily_location(
    "2016-01-01",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)
dl_present = flowclient.daily_location(
    "2016-01-07",
    aggregation_unit=aggregation_unit,
    daily_location_method="last",
    subscriber_subset=None,
)

Specify parameters for flow query:

In [19]:
flow = flowclient.flows(dl_past, dl_present, aggregation_unit=aggregation_unit)
flow

{'query_kind': 'flows',
 'params': {'from_location': {'query_kind': 'daily_location',
   'params': {'date': '2016-01-01',
    'aggregation_unit': 'admin1',
    'daily_location_method': 'last',
    'subscriber_subset': 'all'}},
  'to_location': {'query_kind': 'daily_location',
   'params': {'date': '2016-01-07',
    'aggregation_unit': 'admin1',
    'daily_location_method': 'last',
    'subscriber_subset': 'all'}},
  'aggregation_unit': 'admin1'}}

### Set query running

In [20]:
query_id = flowclient.run_query(conn, flow)
query_id

75c50eec9c84952bb1f578fc67ca8630


### Check status of query

In [21]:
flowclient.get_status(conn, query_id)

(True, <Response [303]>)

### Get query result

In [22]:
df = flowclient.get_result_by_query_id(conn, query_id)
df.head()

,count,name_from,name_to
0,12,Central Development Region,Western Development Region
1,16,Central Development Region,Mid-Western Development Region
2,2,Central Development Region,Far-Western Development Region
3,4,Central Development Region,Eastern Development Region
4,5,Central Development Region,Central Development Region
